# Testing the likelihood external file

To start playing with this notebook, install the ```Likelihood Package``` and its modules according to README.
Alternatively you may be ready to run if ```Cobaya``` and ```CAMB/CLASS``` are installed.

Cobaya documentation: https://cobaya.readthedocs.io/en/latest/

In [1]:
#General imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import interpolate 
import time 

plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=15)
#plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=17)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

### Specific imports

In [2]:
#Import cobaya -need to be installed
import cobaya
#Import external loglike from the Likelihood Package within cobaya interface module
from likelihood.cobaya_interface import EuclidLikelihood

### Step 1: create your dictionary with general specifications

In ```COBAYA```, 'ini files' are 'yaml files', which are in reality understood by Cobaya as python dictionaries. I have added comments explaining each of the lines of the dictionary.


In [5]:
#ATTENTION: CHANGE THIS TO YOUR LOCAL PATH where your external codes are installed: CAMB, polychord, likelihoods...
modules_path = "/data2/cobaya_modules/"

h=0.67
# Info is the 'ini file' dictionary. 
# You can translate this info-dictionary into a yaml file and run: cobaya-run info.yaml such as CosmoMC like

info = {
    #Which parameters would you like to sample?
    'params': {
        # Fixed
        'ombh2': 0.05*h**2, 'omch2': 0.268563*h**2, 'H0': 67, 'tau': 0.0925,
        'mnu': 0.06, 'nnu': 3.046, 'As': 2.12605e-9,
        # Sampled - just as an example we assume we will be sampling over ns
        'ns': 0.96, #{'prior': {'min': 0.8, 'max': 1.2}},
        #To be passed to euclid which likelihood to use (preliminary)
        # 1: shear
        # 2: spec
        # 12: both
        'like_selection': 12,
        # Bias parameters
        # Photo IST:F values
        'b1_photo': 1.0997727037892875,
        'b2_photo': 1.220245876862528,
        'b3_photo': 1.2723993083933989,
        'b4_photo': 1.316624471897739,
        'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918,
        'b7_photo': 1.4446452851824907,
        'b8_photo': 1.4964959071110084,
        'b9_photo': 1.5652475842498528,
        'b10_photo': 1.7429859437184225,
        # Spec IST:F values
        'b1_spec': 1.46,
        'b2_spec': 1.61,
        'b3_spec': 1.75,
        'b4_spec': 1.90},
    #Which theory code you want to use? HERE CAMB
    'theory': {'camb': {'stop_at_error': True, 'extra_args':{'num_massive_neutrinos': 1}}},
    #Which sample do you want to use? HERE I use MCMC for testing
    'sampler': {'mcmc': None},  
    #Where have you installed your modules (i.e: CAMB, polychord, likelihoods...)
    'packages_path': modules_path,
    #Where are the results going to be stored?
    'output': 'chains/my_euclid_experiment',
    #Likelihood: we load the likelihood as an external function
    'likelihood': {'Euclid': EuclidLikelihood},
    'debug': False,
    'force': True
    }

### Step 2: Call the 'model' wrapper of ```COBAYA```

In principle, we could run directly cobaya with the above dictionary and start producing chains. However, for testing purposes, the 'model' wrapper is created, which allows to enter directly to theory code's quantities. It works similarly as an 'evaluator'.

I use it here to give a feeling of what ```COBAYA``` is doing internally. Advice: good for debbuging too.

In [6]:
# Model wrapper of cobaya
from cobaya.model import get_model
t1 = time.time()
model = get_model(info)
t2 = time.time()
print(t2-t1)

[model] *WARNING* Ignored blocks/options: ['sampler', 'output', 'force']
[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[camb] Importing *local* CAMB from /data2/cobaya_modules/code/CAMB
[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[camb] Importing *local* CAMB from /data2/cobaya_modules/code/CAMB
2.56280255317688


In [7]:
# Which theory requirements is the likelihood asking to the Boltzman Solver?
print('\n Requirements \n')
print(model.provider.requirement_providers)
# So, what have we requested?
print('\n Requested \n')
print(model.requested())


 Requirements 

{'Pk_interpolator': camb, 'comoving_radial_distance': camb, 'angular_diameter_distance': camb, 'Hubble': camb, 'sigma_R': camb, 'fsigma8': camb, 'CAMB_transfers': camb.transfers}

 Requested 

{'camb.transfers': [{'CAMB_transfers':{'non_linear': False, 'needs_perts': True}}], 'camb': [{'Pk_interpolator':{}}, {'comoving_radial_distance':{'z': array([0.        , 0.04040404, 0.08080808, 0.12121212, 0.16161616,
       0.2020202 , 0.24242424, 0.28282828, 0.32323232, 0.36363636,
       0.4040404 , 0.44444444, 0.48484848, 0.52525253, 0.56565657,
       0.60606061, 0.64646465, 0.68686869, 0.72727273, 0.76767677,
       0.80808081, 0.84848485, 0.88888889, 0.92929293, 0.96969697,
       1.01010101, 1.05050505, 1.09090909, 1.13131313, 1.17171717,
       1.21212121, 1.25252525, 1.29292929, 1.33333333, 1.37373737,
       1.41414141, 1.45454545, 1.49494949, 1.53535354, 1.57575758,
       1.61616162, 1.65656566, 1.6969697 , 1.73737374, 1.77777778,
       1.81818182, 1.85858586, 1.898

In [11]:
# To execute the likelihood, you need to make an evaluation of the posterior
# Attention: this chi2 for the moment, does not make sense yet
# if ns is sampled, we need to obtain a value from the prior
# point = dict(zip(model.parameterization.sampled_params(),
#                 model.prior.sample(ignore_external=True)[0]))
# As all parameters are fixed, just call the posterior
t1 = time.time()
model.logposterior({})
t2 = time.time()
print(t2-t1)

0.0002455711364746094


In [ ]:
# which parameters values does the model have?
print('\n Parameters \n')
print(model.provider.params)

### Step 3: Call the sampler of  ```COBAYA```

Let's check if ```cobaya```works properly with samplers

In [ ]:
# for the moment it is commented out as we do not want to run the sampler
# from cobaya.run import run
# info_updated, samples = run(info)

### Step 4: Cosmology module and ```COBAYA```

In principle, the cobaya interface only pass cosmology parameters (for instance $\Omega_m$, $\Omega_b$, $H$) and ask for theory requirements from the boltzmann solver. Internally, the likelihood module uses the cosmology class to deal with the different GCspec, GCphot, WL recipes.

In [ ]:
#Import Cosmology module from the Likelihood Package to play with cosmological quantities
from likelihood.cosmo.cosmology import Cosmology

In [ ]:
# Some of the theory needs require extra info (redshift, ks)...
z_min = 0.0
z_max = 4.0
z_samp = 100
z_win = np.linspace(z_min, z_max, z_samp)
k_max = 0.2

In [ ]:
# Cobaya_interface save the cosmology parameters and the cosmology requirements 
# from CAMB/CLASS via COBAYA to the cosmology class

# This dictionary collects info from cobaya
theory_dic = {'H0': model.provider.get_param('H0'),
                  'omch2': model.provider.get_param('omch2'),
                  'ombh2': model.provider.get_param('ombh2'),
                  'mnu': model.provider.get_param('mnu'),
                  'comov_dist': model.provider.get_comoving_radial_distance(z_win),
                  'angular_dist': model.provider.get_angular_diameter_distance(z_win),
                  'H': model.provider.get_Hubble(z_win),
                  'Pk_interpolator': model.provider.get_Pk_interpolator(nonlinear=False),
                  'Pk_delta': None,
                  'fsigma8': None,
                  'z_win': z_win
                  }
theory_dic['Pk_delta'] = model.provider.get_Pk_interpolator(("delta_tot", "delta_tot"), nonlinear=False)
theory_dic['fsigma8'] = model.provider.get_fsigma8(z_win)
# Remember: h is hard-coded
R, z, sigma_R = model.provider.get_sigma_R()
#print(R)
theory_dic['sigma_8'] = sigma_R[:, 0]

In [ ]:
# Initialize cosmology class from likelihood.cosmo.cosmology
# By default: LCDM
t1 = time.time()
cosmology = Cosmology()
t2 = time.time()
print(t2-t1)

print("Previous dic\n\n:", cosmology.cosmo_dic)

In [ ]:
# the cobaya_interface.py save cobaya theory dictionary as above into cosmology.cosmo_dic
t1 = time.time()
cosmology.cosmo_dic.update(theory_dic)
t2 = time.time()
print(t2-t1)


t1 = time.time()


cosmology.update_cosmo_dic(z_win, 0.005)
t2 = time.time()
print(t2-t1)



print("Updated dic\n\n:", cosmology.cosmo_dic)

In [ ]:
# Cosmology class calculates other cosmology quantities needed by GC, WL...
# Growth factor
D=cosmology.growth_factor(z_win, 0.002)
print('D(z, 0.002) =', D)
# Growth rate
f=cosmology.growth_rate(z_win, 0.002)
print('f(0.0, 0.002) =', f(0.0))

#### Read data/ExternalBenchmark

In [ ]:
comov_EB = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/ComDist-LCDM-Lin-noIA.dat")
Pm_EB = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/Pnl-TB-LCDM.dat")
ni_EB = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/niTab-EP10-RB00.dat")
Wi_EB = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/WiGamma-LCDM-Lin-noIA.dat")
Wi_EB_G = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/WiG-LCDM-Lin-noIA.dat")
C_GC = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/CijGG-LCDM-Lin-noIA.dat")
C_LL = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Photometric/CijLL-LCDM-Lin-noIA.dat")
Pm_ISTF = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Pm_z0.0_ISTF.dat")

In [ ]:
bias_dic = np.load("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/matteo_bias.npy", allow_pickle = True).item()

In [ ]:
z_extended = np.linspace(2.6, 4, 100)
np.concatenate([bias_dic['z'], z_extended])

In [ ]:
z_extended = np.linspace(2.6, 4, 100)
np.concatenate([bias_dic['bias'], bias_dic['bias'][-1]*np.ones(len(z_extended))])

In [ ]:
# Make panda DataFrame for Pm to filter redshift
import pandas as pd
Pm_pd_EM = pd.DataFrame(Pm_EB, columns =["logk", "z", "logPnl", "logPlin"])
z_values = Pm_pd_EM['z']
unique_z = np.unique(z_values)
#unique_z

## Cosmology plots

In [ ]:
# Matter power spectrum
plt.figure(figsize=(10,6));
ks=10**(Pm_pd_EM[Pm_pd_EM['z']==2.5]['logk'])
plt.loglog(ks, cosmology.cosmo_dic['Pk_delta'].P(z_win[0], ks), label=r"Cobaya z={}".format(z_win[0]))
plt.loglog(ks, 10**Pm_pd_EM[Pm_pd_EM['z']==0.0]['logPlin'], 'g--', label=r"Benchmark")
plt.loglog(Pm_ISTF[:, 0], Pm_ISTF[:, 1], 'r-.', label=r"IST:F")


plt.xlabel("k");
plt.ylabel(r"$P_m$");
#plt.xlim(10**(-4), 0.2)
plt.ylim(10**(2), 5*10**(6))
plt.legend();
plt.figure(figsize=(10,6));
plt.loglog(ks, 100*(cosmology.cosmo_dic['Pk_delta'].P(z_win[0], ks)-10**Pm_pd_EM[Pm_pd_EM['z']==z_win[0]]['logPlin'])/(10**Pm_pd_EM[Pm_pd_EM['z']==z_win[0]]['logPlin']), 'g--', label=r"Benchmark")

plt.xlim(10**(-4), 10);
plt.ylim(10**(-5), 50);
plt.xlabel("k");
plt.ylabel(r"$\Delta P_m /P_m $ (%)");

In [ ]:
plt.figure(figsize=(10,6))
for z in unique_z[:10]:
    # Matter power spectrum
    #print(z)
    ks=10**(Pm_pd_EM[Pm_pd_EM['z']==z]['logk'])
    plt.loglog(ks, 100*(cosmology.cosmo_dic['Pk_delta'].P(z, ks)-10**Pm_pd_EM[Pm_pd_EM['z']==z]['logPlin'])/(10**Pm_pd_EM[Pm_pd_EM['z']==z]['logPlin']),label=r"z={}".format(z))
    plt.xlabel("k");
    plt.ylabel(r"Relative $P_m$ (\%)");
    plt.legend();
    plt.xlim(10**(-4), 0.2)
    plt.ylim(10**(-5), 30)

In [ ]:
# It seems that there is a bug in the calculation of D and f. Let's verify with CLASS
from classy import Class
cosmo_class = Class()
cosmo_class.set({'omega_b':0.05*h**2,'omega_cdm':0.268563*h**2,
                 'h':0.67,'A_s':2.12605e-9,'n_s':0.96,'tau_reio':0.0925, 'Omega_Lambda': 0.68,
                 'N_ur': 2.046})
cosmo_class.compute();
class_bg = cosmo_class.get_background();

In [ ]:
# Read IST:F data
data_ist = np.genfromtxt("/data2/fG_Growth_IST_9bins.txt")
comov_ist_Mpc = np.genfromtxt("/data2/ComovingDistance_Mpc_ISTF.txt")

In [ ]:
# Plot Growth factor, growth rate, sigma8
plt.figure(figsize=(10,6))
plt.plot(z_win, D, label="Self-computed (CAMB)")
plt.plot(data_ist[:, 0], data_ist[:, 2], 'r^', markersize="12", label="IST:F")
plt.xlabel("z")
plt.ylabel("D(z)")
for i, zs in enumerate(np.linspace(0, 2.5, 50)):
    plt.plot(zs, cosmo_class.scale_independent_growth_factor(zs), 'ko', label="CLASS" if i == 0 else "")
plt.legend(loc=1);
#plt.xlim(0, 4)

In [ ]:
# Growth rate
plt.figure(figsize=(10,6))
plt.plot(data_ist[:, 0], data_ist[:, 1], 'r^', markersize="12", label="IST:F" )
#for i, zs in enumerate(np.linspace(0, 2.5, 50)):
#    plt.plot(zs, cosmo_class.scale_independent_growth_factor_f(zs), 'o',
#             color='k', label="CLASS" if i == 0 else "")
plt.plot(np.linspace(0, 2.5, 500), f(np.linspace(0, 2.5, 500)), label="Self-computed (CAMB)")
plt.plot(z_win, theory_dic['fsigma8']/theory_dic['sigma_8'], label=r"COBAYA (CAMB): $f\sigma_8/\sigma_8$")
plt.xlabel("z")
plt.ylabel(r"$f(z)$")
plt.legend();

In [ ]:
# Growth rate
plt.figure(figsize=(10,6))
plt.plot(z_win, 100*(f(z_win)-cosmology.cosmo_dic['fsigma8_z_func'](z_win)/cosmology.cosmo_dic['sigma8_z_func'](z_win))/f(z_win), 'o-', label='Difference Self-computed and COBAYA')
plt.plot(data_ist[:, 0], 100*(f(data_ist[:, 0])-data_ist[:, 1])/f(data_ist[:, 0]), 'r^', markersize="12", label='Difference Self-computed and IST:F')
plt.plot(data_ist[:, 0], 100*(cosmology.cosmo_dic['fsigma8_z_func'](data_ist[:, 0])/cosmology.cosmo_dic['sigma8_z_func'](data_ist[:, 0])-data_ist[:, 1])/data_ist[:, 1], 'o-', label='Difference COBAYA and IST:F')


plt.xlabel("z")
plt.ylabel(r"$\Delta f(z)/f(z) (\%)$")
plt.legend()

In [ ]:
# TODO: compare with Santi

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(z_win, cosmology.cosmo_dic['sigma_8'], 'bo', markevery=10, label="real sigma8 (COBAYA)")
plt.plot(z_win, cosmology.cosmo_dic['sigma8_z_func'](z_win), 'b-', label='interpolated sigma8')
plt.plot(z_win, cosmology.cosmo_dic['fsigma8'], 'go', markevery=10, label="real sigma8 (COBAYA)")
plt.plot(z_win, theory_dic['fsigma8'], 'g-', label="fsigma8 (z) (COBAYA)")
plt.legend()
plt.xlabel("z");

In [ ]:
plt.plot(z_win, cosmology.cosmo_dic['H_z_func'](z_win),
         label='interpolated H')
plt.plot(z_win, cosmology.cosmo_dic['H'], 'bo', 
        markevery=10,
         label='real H')
plt.legend()
plt.xlabel("z")
plt.xlabel("H(z)");

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(z_win, cosmology.cosmo_dic['r_z_func'](z_win), label='Interpolated comoving distance')
plt.plot(z_win, cosmology.cosmo_dic['comov_dist'], 'b*', markevery=10, label='Cobaya comoving distance')
plt.plot(class_bg['z'], class_bg['comov. dist.'], 'ko', label="CLASS", markevery=10,)    
plt.plot(comov_EB[:, 0], comov_EB[:, 1], 'g--', label=r"Benchmark")
plt.plot(comov_ist_Mpc[:, 0], comov_ist_Mpc[:, 1], 'r^', markersize="12", label="IST:F", markevery=20)
plt.legend()
plt.ylabel("Comoving Distance");
plt.xlabel("z");
plt.xlim(0, 3.5);


In [ ]:
# Attention: we want in reality the comoving angular distance!
plt.figure(figsize=(10,6))
plt.plot(z_win, cosmology.cosmo_dic['d_z_func'](z_win), 'g-', label='Interpolated Angular distance')
plt.plot(z_win, cosmology.cosmo_dic['angular_dist'], 'go', markevery=10, label='Cobaya Angular distance')
plt.legend()
plt.xlabel("Angular Diameter Distance");
plt.xlabel("z");

### Step 5: Galdist, Shear modules and ```COBAYA```



In [ ]:
from likelihood.general_specs.estimates import Galdist

In [ ]:
plt.figure(figsize=(22,6))
color = ['red', ]
z_phot = np.linspace(0.0100, 2.5, 1000)
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    galdist = Galdist(i)
    plt.plot(z_phot, galdist.n_i(z_phot), label="i = {}".format(i))
    plt.plot(ni_EB[:, 0], ni_EB[:, i], 'o', markevery=10)
plt.legend()
plt.xlim(0, 2.9);
plt.ylabel("n(z)", fontsize=30);
plt.xlabel("z", fontsize=30);

In [ ]:
# Vincenzo's data is still not normalized!

In [ ]:
plt.figure(figsize=(22,6))
z_phot = np.linspace(0.0100, 2.5, 1000)
plt.title("Benchmark", fontsize=30)
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    galdist = Galdist(i)
    #plt.plot(z_phot, galdist.n_i(z_phot), label="i = {}".format(i))
    plt.plot(ni_EB[:, 0], ni_EB[:, i], 'o', markevery=10, label="i = {}".format(i))
plt.xlim(0, 2.5);
plt.ylabel("n(z)", fontsize=30);
plt.xlabel("z", fontsize=30);
plt.legend();

In [ ]:
# Try to see if the problem if Vicenzo's normalization 
ni_v_inter = interpolate.InterpolatedUnivariateSpline(ni_EB[:, 0], ni_EB[:, 1], ext=2)
denom = integrate.quad(ni_v_inter, ni_EB[:, 0][0], ni_EB[:, 0][-1])
print(denom[0])
# Effectively, Vincenzo's is not normalized

In [ ]:
plt.figure(figsize=(22,6))
color = ['red', 'green', 'pink', 'black', 'purple', 'grey', 'orange', 'blue', 'darkblue', 'salmon']
z_phot = np.linspace(0.0100, 2.5, 1000)
plt.title("Estimates", fontsize=30)
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    
    galdist = Galdist(i)
    ni_v_inter = interpolate.InterpolatedUnivariateSpline(ni_EB[:, 0], ni_EB[:, i], ext=2)
    denom = integrate.quad(ni_v_inter, ni_EB[:, 0][0], ni_EB[:, 0][-1])
    t1 = time.time()
    galdist.n_i(z_phot)
    t2 = time.time()
    print(t2-t1)
    
    plt.plot(z_phot, galdist.n_i(z_phot), 'o', markevery=20, label="i = {}".format(i), color=color[i-1])
    plt.plot(ni_EB[:, 0], ni_EB[:, i]/denom[0], '-', color=color[i-1])
plt.xlim(0, 2.5);
plt.ylabel("n(z)", fontsize=30);
plt.xlabel("z", fontsize=30);
plt.legend();

### Problems:
 * Slightly difference in the self-computed limits of estimates.py and the Benchmark ones
 * Benchmark is not normalized

In [ ]:
from likelihood.photometric_survey.shear import Shear
shear = Shear(cosmology.cosmo_dic)

In [ ]:
from likelihood.photometric_survey.shear import Shear
shear = Shear(cosmology.cosmo_dic)
plt.figure(figsize=(22,6))
plt.title("Shear", fontsize=30)
t1 = time.time()
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    plt.plot(Wi_EB[:, 0], Wi_EB[:, i], '-', markevery=4, color=color[i-1])
    print(i)
    for j, z in enumerate(np.linspace(0.01, 2.5, 100)):
        shear.WL_window(z, i)
        plt.plot(z, shear.WL_window(z, i), 'o-', color=color[i-1], label="i={}".format(i) if j == 0 else "")
t2 = time.time()
print(t2-t1)
plt.legend()
plt.ylabel(r"$W_\gamma(z)$", fontsize=30);
plt.xlabel("z", fontsize=30);

In [ ]:
from likelihood.photometric_survey.shear import Shear
shear = Shear(cosmology.cosmo_dic)
plt.figure(figsize=(22,6))
plt.title("Shear", fontsize=30)
t1 = time.time()
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print(i)
    #plt.plot(Wi_EB_G[:, 0], Wi_EB_G[:, i], '-', markevery=4, color=color[i-1])
    for j, z in enumerate(np.linspace(0.01, 2.5, 100)):
        plt.plot(z, shear.GC_window(0.5, z, i), 'o', color=color[i-1], label="i={}".format(i) if j == 0 else "")
t2 = time.time()
print(t2-t1)
plt.legend()
plt.ylabel(r"$W_{GC}(z)$", fontsize=30);
plt.xlabel("z", fontsize=30);

In [ ]:
from likelihood.photometric_survey.shear import Shear
shear = Shear(cosmology.cosmo_dic)
plt.figure(figsize=(22,6))
plt.title("Shear", fontsize=30)
t1 = time.time()
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print(i)
    plt.plot(Wi_EB_G[:, 0], Wi_EB_G[:, i], '-', markevery=4, color=color[i-1])
t2 = time.time()
print(t2-t1)
plt.legend()
plt.ylabel(r"$W_{GC}(z)$", fontsize=30);
plt.xlabel("z", fontsize=30);

## Fiducial model

In [ ]:
# Initialize cosmology class from likelihood.cosmo.cosmology
# By default: LCDM
fiducial_cosmology = Cosmology()
print(fiducial_cosmology.cosmo_dic)
print(fiducial_cosmology.cosmo_dic['ombh2'])

In [ ]:
info_fiducial = {
    #Which parameters would you like to sample?
    'params': {
        # Fixed
        'ombh2': fiducial_cosmology.cosmo_dic['ombh2'], 
        'omch2': fiducial_cosmology.cosmo_dic['omch2'], 
        'H0': fiducial_cosmology.cosmo_dic['H0'], 
        'tau': fiducial_cosmology.cosmo_dic['tau'],
        'mnu': fiducial_cosmology.cosmo_dic['mnu'], 
        'nnu': fiducial_cosmology.cosmo_dic['nnu'],
        'ns': fiducial_cosmology.cosmo_dic['ns'],
        'As': fiducial_cosmology.cosmo_dic['As'],
        },
    #We should decide with Boltzmann we want for the fiducial: CAMB
    'theory': {'camb': {'stop_at_error': True, 'extra_args':{'num_massive_neutrinos': 1}}},
    #Where have you installed your modules (i.e: CAMB, polychord, likelihoods...)
    'packages_path': modules_path,
    #Likelihood: we load the likelihood as an external function
    'likelihood': {'one': None}
    }

In [ ]:
model_fiducial = get_model(info_fiducial)

# Declare our desired theory product
# (there is no cosmological likelihood doing it for us)
model_fiducial.add_requirements({"Pk_interpolator": {"z": z_win,
                                    "k_max": k_max,
                                    "nonlinear": False,
                                    "vars_pairs": ([["delta_tot",
                                                     "delta_tot"]])},
                "comoving_radial_distance": {"z": z_win},
                "angular_diameter_distance": {"z": z_win},
                "Hubble": {"z": z_win, "units": "km/s/Mpc"},
                "fsigma8": {"z": z_win, "units": None}})

# Compute and extract the CMB power spectrum
# (In muK^-2, without l(l+1)/(2pi) factor)
# notice the empty dictionary below: all parameters are fixed
model_fiducial.logposterior({})


In [ ]:
fiducial_cosmology.cosmo_dic['z_win']=z_win
fiducial_cosmology.cosmo_dic['comov_dist']= model_fiducial.provider.get_comoving_radial_distance(z_win),
fiducial_cosmology.cosmo_dic['angular_dist']= model_fiducial.provider.get_angular_diameter_distance(z_win),
fiducial_cosmology.cosmo_dic['H'] = model_fiducial.provider.get_Hubble(z_win),
fiducial_cosmology.cosmo_dic['Pk_interpolator'] = model_fiducial.provider.get_Pk_interpolator(nonlinear=False),
fiducial_cosmology.cosmo_dic['Pk_delta']= model_fiducial.provider.get_Pk_interpolator(("delta_tot", "delta_tot"), nonlinear=False)
fiducial_cosmology.cosmo_dic['fsigma8'] = model_fiducial.provider.get_fsigma8(z_win)

In [ ]:
fiducial_cosmology.interp_H()
fiducial_cosmology.interp_comoving_dist()
fiducial_cosmology.interp_angular_dist()
#print(fiducial_cosmology.cosmo_dic)

In [ ]:
fiducial_cosmology.cosmo_dic['d_z_func'](0.0000001)

In [ ]:
h = fiducial_cosmology.cosmo_dic['H'][0]/100
h[0]

### Step 6: Spec module and ```COBAYA```

In [ ]:
from likelihood.spectroscopic_survey.spec import Spec

In [ ]:
spec = Spec(cosmology.cosmo_dic, fiducial_cosmology.cosmo_dic)

In [ ]:
k_win = np.linspace(0.01, 0.33, 100)

In [ ]:
spec_data = np.genfromtxt("/data2/EUCLID/likelihood-implementation/data/ExternalBenchmark/Spectroscopic/data_Ariel.dat")
vector = spec_data
def inter_pg0(ellx):
    #ellx  = 1, 3, 5
    ks = vector[:,0]
    pks = vector[:,ellx]
    return interpolate.interp1d(np.log(ks*h[0]),np.log(pks/h[0]**3),kind='cubic',fill_value='extrapolate')

def pg0(ellx,k): #DS Takes true k
    return np.exp(inter_pg0(ellx)(np.log(k)))

In [ ]:
color=['blue', 'red', 'green']
plt.figure(figsize=(22,6))
#plt.plot(k_win, k_win*P0, '*', color=color[0], markersize=13, label='no AP')
#plt.plot(k_win, k_win*P2, '*', color=color[1], markersize=13)
#plt.plot(k_win, k_win*P4, '*', color=color[2], markersize=13)
for i, k in enumerate(k_win):
    for j, mu in enumerate([0, 2, 4]):
        plt.plot(k, k*spec.multipole_spectra(1.0, k, mu), 'o', color=color[j],
            label=r"$z=1.00; \ell={}$".format(mu) if i == 0 else "")
        plt.plot(k, (2*np.pi)**3*k*pg0(mu+1, k), marker='*', linestyle = '-', label="Benchmark " if i == 0 else "", color=color[j])
plt.xlabel("$k$", fontsize=30);
plt.ylabel(r"$k*P_\ell$", fontsize=30);
plt.legend();
plt.title("Spectroscopic Probe", fontsize=24)

In [ ]:
color=['blue', 'red', 'green']
plt.figure(figsize=(22,6))

for i, k in enumerate(k_win):
    for j, mu in enumerate([0, 2, 4]):
        plt.plot(k, 100*(k*spec.multipole_spectra(1.0, k, mu)-(2*np.pi)**3*k*pg0(mu+1, k))/(2*np.pi)**3*k*pg0(mu+1, k), 'o-', color=color[j],
            label=r"$z=1.00; \ell={}$".format(mu) if i == 0 else "")
plt.xlabel("$k$", fontsize=30);
plt.ylabel(r"$\Delta(k*P_\ell)/(k*P_\ell)   (\%)$", fontsize=30);
plt.legend();
plt.title("Spectroscopic Probe", fontsize=24)

In [ ]:
color=['blue', 'red', 'green']
plt.figure(figsize=(22,6))

for i, k in enumerate(k_win):
    for j, mu in enumerate([0]):
        #plt.plot(k, 100*(k*spec.multipole_spectra(1.0, k, mu)-pg0(mu,k))/pg0(mu,k), 'o-', color=color[j],
        #    label=r"$z=1.00; \ell={}$".format(mu) if i == 0 else "")
        plt.plot(k, k*pg0(mu,k), 'o')
plt.xlabel("$k$", fontsize=30);
plt.ylabel(r"$\Delta(k*P_\ell)/(k*P_\ell)   (\%)$", fontsize=30);
plt.legend();
plt.title("Spectroscopic Probe", fontsize=24)

In [ ]:
pg0(ellx,k)